In [ ]:
import numpy as np
from scipy.io import loadmat
from sklearn.linear_model import LogisticRegression as LR
import matplotlib.pyplot as plt
%matplotlib inline

# Theano imports
import theano
theano.config.floatX = 'float32'
import theano.tensor as T

# Plotting utility
from utils import tile_raster_images as tri

# The dataset
The dataset is the mnist digits which is a common toy data set for testing machine learning methods on images. This is a subset of the mnist set which have also been shrunked in size. Let's load them and plot some. In addition to the images, there are also the labels: 0-9 or even-odd.

Load the data.

In [ ]:
data = loadmat('small_mnist.mat')

# Training data (images, 0-9, even-odd)
# Images are stored in a (batch, x, y) array
# Labels are integers
train_im = data['train_im']
train_y = data['train_y'].ravel()
train_eo = data['train_eo'].ravel()

# Validation data (images, 0-9, even-odd)
# Same format as training data
valid_im = data['valid_im']
valid_y = data['valid_y'].ravel()
valid_eo = data['valid_eo'].ravel()

Plot 10 of the training images

In [ ]:
im_size = train_im.shape[-1]
ims = tri(train_im[:10].reshape((-1, im_size**2)), (im_size, im_size), (1, 10), (1,1))
plt.imshow(ims, cmap='gray', interpolation='nearest')
plt.axis('off')
print('Labels: {}'.format(train_y[:10]))
print('Odd-Even: {}'.format(train_eo[:10]))

## Baseline linear classifier
Before we spend our precious time setting up and training deep networks on the data, let's see how a simple linear classifier from sklearn can do.

In [ ]:
# Create the classifier to do multinomial classification
linear_classifier = LR(solver='lbfgs', multi_class='multinomial')

In [ ]:
# Train and evaluate the classifier
linear_classifier.fit(train_im.reshape(-1, im_size**2), train_y)
print('Training Error on (0-9): {}'.format(linear_classifier.score(train_im.reshape(-1, im_size**2), train_y)))
print('Validation Error on (0-9): {}'.format(linear_classifier.score(valid_im.reshape(-1, im_size**2), valid_y)))

Try training a linear classifier on the Even-Odd labels: train_eo!!!!!

# Using a Deep Nets library
If you're just starting off with deep nets and want to quickly try them on a dataset it is probably easiest to start with an existing library rather than writing your own. There are now a bunch of different libraries written for Python. We'll be using Keras which is designed to be easy to use. In matlab, there is the Neural Network Toolbox.

We'll do the next most complicated network comparer to linear regression: a two layer network!

In [ ]:
# Import things from Keras Library
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

This is a simple network made from two layers.

In [ ]:
# Create the network!
model = Sequential()

# First fully connected layer
model.add(Dropout(0.0, input_shape=(im_size**2,)))
model.add(Dense(im_size**2))
model.add(Activation('tanh'))

# Second fully connected layer with softmax output
model.add(Dropout(0.0))
model.add(Dense(10))
model.add(Activation('softmax'))

# Adam is a simple optimizer, SGD has more parameters but may give better results
#opt = Adam()
#opt = RMSprop()
opt = SGD(lr=0.1, momentum=0.9, decay=0.0001, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_im.reshape(-1, im_size**2), np_utils.to_categorical(train_y), nb_epoch=20, batch_size=100)
print model.evaluate(train_im.reshape(-1, im_size**2), np_utils.to_categorical(train_y), batch_size=100)
print model.evaluate(valid_im.reshape(-1, im_size**2), np_utils.to_categorical(valid_y), batch_size=100)

# Junk

In [ ]:
from pylearn2.datasets.mnist import MNIST
from scipy.io import savemat

In [ ]:
ds = MNIST('train')
ts = MNIST('test')
train_size = 10000
valid_size = 1000
offset = 1
jump = 2
crop = 2

In [ ]:
trX = ds.X.reshape(-1, 28, 28)
trX = (trX[:, ::jump, ::jump] + trX[:, offset::jump, offset::jump])/2.
trX = trX[:, crop:-crop, crop:-crop]
teX = ts.X.reshape(-1, 28, 28)
teX = (teX[:, ::jump, ::jump] + teX[:, offset::jump, offset::jump])/2.
teX = teX[:, crop:-crop, crop:-crop]

train_X = trX[:train_size]
train_y = ds.y[:train_size]
valid_X = teX[-valid_size:]
valid_y = ts.y[-valid_size:]
train_yp = (train_y %2).ravel()
valid_yp = (valid_y %2).ravel()

#valid_X -= train_X.mean(axis=0)
#train_X -= train_X.mean(axis=0)

In [ ]:
data = {'train_im': train_X,
        'train_y': train_y,
        'valid_im': valid_X,
        'valid_y': valid_y,
        'train_eo': train_yp,
        'valid_eo': valid_yp}
savemat('small_mnist.mat', data)

In [ ]:
ls -rtlh